In [38]:
from collections import deque
import pickle

def select_root_with_minimum_m(G, k):
    m_values = {}
    
    for v in range(len(G)):
        m_v = 0
        while len(get_N_h(G, v, m_v)) < k:
            m_v += 1
        m_values[v] = m_v
    return min(m_values, key=m_values.get)

def children_of(G, v, visited):
    return [node for node in G[v] if node not in visited]

def get_N_h(G, v, h):
    visited = set()
    queue = deque([(v, 0)])

    while queue:
        node, depth = queue.popleft()
        if node not in visited:
            visited.add(node)
            if depth < h:
                for neighbor in G[node]:
                    if neighbor not in visited:
                        queue.append((neighbor, depth + 1))
    return visited

def r_m_values(G, labels_assigned, k):
    r_values = {}
    m_values = {}

    for v in range(len(G)):
        r_v = 0
        m_v = 0
        while len(set(labels_assigned[u] for u in get_N_h(G, v, r_v))) < k:
            r_v += 1
        r_values[v] = r_v

        while len(get_N_h(G, v, m_v)) < k:
            m_v += 1
        m_values[v] = m_v

    return r_values, m_values

# def tree_node_labelling(G, k):
#     K = list(range(k))
#     r = select_root_with_minimum_m(G, k)
#     BFS_queue = deque([(r, [])])
#     labels_assigned = {}
#     labels_used = [False] * len(K)

#     while BFS_queue:
#         v, visited = BFS_queue.popleft()

#         if all(labels_used):
#             labels_used = [False] * len(K)

#         if v != r:
#             m_v = 0
#             while len(get_N_h(G, v, m_v)) < k:
#                 m_v += 1
#             available_labels = list(set(K) - set(labels_assigned[u] for u in get_N_h(G, v, m_v) if u in labels_assigned))
#         else:
#             available_labels = [label for i, label in enumerate(K) if not labels_used[i]]


#         # In case available_labels is empty after resetting labels_used
#         if not available_labels:
#             available_labels = [label for i, label in enumerate(K) if not labels_used[i]]

#         best_label = available_labels.pop()
#         labels_assigned[v] = best_label
#         labels_used[K.index(best_label)] = True

#         visited.append(v)
#         for child in children_of(G, v, visited):
#             BFS_queue.append((child, visited.copy()))

#     return labels_assigned

def dfs(G, visited, node, labels_assigned, K, k):
    visited.add(node)
    m_v = 0
    while len(get_N_h(G, node, m_v)) < k:
        m_v += 1

    # Assign label
    if node in labels_assigned:
        available_labels = [labels_assigned[node]]
    else:
        available_labels = list(set(K) - set(labels_assigned[u] for u in get_N_h(G, node, m_v-1) if u in labels_assigned))

    if not available_labels:
        available_labels = K
    label = min(available_labels)
    labels_assigned[node] = label

    for child in G[node]:
        if child not in visited:
            dfs(G, visited, child, labels_assigned, K, k)

def tree_node_labelling(G, k):
    K = list(range(k))
    r = select_root_with_minimum_m(G, k)
    labels_assigned = {}
    visited = set()

    dfs(G, visited, r, labels_assigned, K, k)

    return labels_assigned



# Load input instances
trees = pickle.load(open("Examples_of_AdjLists_of_Trees", "rb"))
k_values = pickle.load(open("Examples_of_k_values", "rb"))

# Compute solutions
solutions = []

# Compute proximity ratio
max_ratios = []
index = 0
for i in range(len(trees)):
    G = trees[i]
    k = k_values[i]
    result = tree_node_labelling(G, k)
    solutions.append(list(result.values()))

    r_values, m_values = r_m_values(G, result, k)
    ratios = [r_values[v] / m_values[v] for v in range(len(G))]
    max_ratios.append(max(ratios))
    print(index,'r value ', r_values)
    print(index, 'm value ', m_values)
    index = index + 1
    
# Save solutions
print('\n')
print(solutions)            
print(max_ratios)
print(max(max_ratios))


KeyboardInterrupt: 